# 機械学習に関するTips

## ○ テーブルデータの予測は深層学習よりもツリー系モデルの方が精度が良いらしい


深層学習がテーブルデータで精度が出ない場合の理由は主に以下二つ。
* 外れ値が含まれる、目的変数の分布が正規分布でないといったことがあるとそれら不正規な部分を過学習してしまう  
  深層学習では標準化が必要であり、(scikit-learnの)標準化は正規分布を前提にしているため上手く標準化できない。  
  下記の参考サイトの非正規分布の標準化をする必要があるかも。
* 予測に役立たない特徴量もしっかり使おうとしてしまう。

参考：
* [元記事](https://exploratory.io/note/kanaugust/XGBoost-knM4aqw2IL)  
* [非正規分布の標準化について](https://biolab.sakura.ne.jp/robust-z-score.html)
* [Kaggleでの優勝回数比較](https://www.datarobot.com/jp/blog/is-deep-learning-almighty/)

## ○ 線形モデルにおいて目的変数と特徴量が正規分布に近いと精度が上がるかも 

元文：  
線形モデルは目標値や特徴量が正規分布である必要はないが、目的変数や特徴量の変換（logなど）によって正規分布に近づけることで、適合の質が向上する場合がある。  
これは、外れ値の影響を減らすことができるからである。強く歪んだ特徴量は、一般的にターゲットと線形関係にない（これは線形モデルの仮定の一つである）。  

特徴量は確かに標準化する際に正規分布を仮定しているため、変換によって良い効果が得られそう。  
目的変数も変換するのは意外だが、確かに外れ値との残差を減らそうと学習するのは汎化性能が落ちてしまいそうな気がする。  
特徴量、目的変数ともにlogに変換した後の特徴量空間における線形回帰の予測は線形に見えるが、変換前の特徴量空間では非線形に見えるようになる。  

参考:  
* [KaggleのKernel](https://www.kaggle.com/code/dejavu23/house-prices-eda-to-ml-beginner/comments)
* [Scikit-Learnのドキュメント](https://scikit-learn.org/stable/modules/preprocessing.html)
* [目的変数をlogに変換する話](https://tomoshige-n.hatenablog.com/entry/2014/08/14/013616)

## ○ 目的変数と特徴量の関係性の調べ方
モデルの精度は、適切なデータの特徴量を加えると向上する。  
逆にノイズとなる特徴量があると、精度が落ちてしまうことがある。    
LightGBMやXGBoostのような「GBDT」は、ノイズとなる特徴量を追加しても精度が落ちづらいと言われているが、  
不要な特徴量は除いておくと、次のメリットがある。[参考](https://potesara-tips.com/lightgbm-feature-importance/)  
・メモリの節約  
・計算時間の短縮。  
　→これにより重要な特徴量を使った特徴量エンジニアリングに時間を回せる

よって、目的変数と特徴量の関係を評価し、適切な特徴量を選定することが重要である。  
特徴量を選定する方法は色々ある。  
* 量的変数の場合  
    * 散布図と相関係数で確認する（人力）          
    目的変数と特徴量で2次元の散布図を書くと概ね傾向、外れ値が分かる。  
    このとき一応ピアソンの相関係数も算出しておくと、相関を定量的に示せる。  
    ただし、ピアソンの相関係数は線形の相関しか評価できないので注意。2次関数の相関だと、0に近い値になる。  
    非線形の相関はMICを使えば評価できるが、導入が面倒なのと特徴量が多いと計算に時間がかかるのが難点。  
    そもそも非線形の相関関係は散布図みれば大体わかるように思う。    
    https://qiita.com/m-hayashi/items/2204e9d7e4e6837c6140
    <br></br>
    * 学習済みモデルに教えてもらう  
    学習済みモデルの予測結果に対する特徴量の寄与を確認する。  
    XGBoost、LightGBMなどの決定木ベースのモデルや線形回帰モデル（Lasso、Ridge等）で使用できる。  
    これらモデルで特徴量重要度を計算した後に、使用する特徴量を決めて他のモデルで学習するということも可能。  
    XGBoostやLightGBMのfeature_importanceを使えば、量的変数だけでなく質的変数の重要度もわかる。  
    GBDTの重要度の指標として、その特徴量が学習の中で使われた回数とするものと、  
    その特徴量による損失関数の減少に寄与した大きさ(gain)を評価する方法があるが、  
    後者を使うのが自然。  
    [特徴量選択手法](https://bakuage.com/ai-blog/entry/2022/05/31/090000)  
    [XGBoostにおける特徴量の重要性評価](https://note.com/y_katayama/n/n5882e9b2d15b)
    <br></br>
    

* 質的変数の場合  
カテゴリ変数（質的変数）であればboxplotなどでカテゴリごとの目的変数の様子を見る。  
カテゴリによって四分位範囲が大きく変わるようであれば、目的変数との相関が強いと言えそう。  
注意点として、カテゴリごとは（順位をつけるようなカテゴリでなければ）基本的に等価であるため、カテゴリの並びは関係ないということ。  
左から右にカテゴリを見て目的変数が増加しているかどうかなどを見ても意味がない。  


## ○ モデルの予測結果と特徴量の関係性の調べ方  
変数重要度とPDPである程度解釈できる模様。  
下記サイトはRの場合で書かれているが、わかりやすかった。  
[変数重要度とPartial Dependence Plotで機械学習モデルを解釈する](https://dropout009.hatenablog.com/entry/2019/01/07/124214)

## ○ 多重共線性（マルチコ）について
マルチコは重回帰モデルのときに起きるので、重回帰以外のモデル（決定木など）では気にしなくてよい。  
https://yolo-kiyoshi.com/2019/05/27/post-1160/  
https://qiita.com/wakichi/items/b68bad8e533dc45d467b

## ○ 2つの特徴量同士の四則演算結果を新たな特徴量にすると、精度が上がるかも？  
データが非現実的で実用性不明。話半分に。  
結局ドメイン知識に基づいて取捨選択し、適用する必要がある。  
https://gri.jp/media/entry/370#f-3a2679f3

## ○ Deep Learning(DL)は万能か？
2020年の記事だが、非常に勉強になった。  
[ディープラーニングは万能なのか](https://www.datarobot.com/jp/blog/is-deep-learning-almighty/)  
* DLでは特徴量抽出もアルゴリズムの中に組み込まれているため、分析者が頑張らなくても良いらしい。
* Kaggleにおいて、非構造化データではDL、構造化データ（テーブルデータ等）ではGBDTの優勝回数が多い。（2017年～2020年）  
* 実務においては、顔認識、医療画像診断、音声アシスタント、機械翻訳ではやはり精度的にDLを使うべきケースが多い。  
    それ以外の分野ではコストも勘案すると、線形モデルやGBDTでも十分な場合が多い。DLは計算コストが高いため。  
    実務ではKaggleのように純粋に精度だけ高めれば良いというわけではなく、コストとのトレードオフを考慮して、  
    最適なモデルを採用する必要がある。

所感：  
やはり、Deep Learning以外もしっかり勉強すべきですな。  
古いものでも沢山アルゴリズムを知っておいて損はないだろう。

## ○ GBDTにおけるearly stoppingの効果
early stoppingを使用する場合と使用せずに決定木の数をチューニングする場合で  
精度、学習時間を比較した記事。  
[Gradient-Boosted Trees: To Early Stop or Not to Early Stop?](https://towardsdatascience.com/gradient-boosting-to-early-stop-or-not-to-early-stop-5ea67ac09d83)  

２ケースの精度に有意な差は見られないが、学習時間はearly stoppingを用いる場合の方が圧倒的に早いと結論付けられている。  
精度同程度で早いならearly stopping是非使おうぜとのこと。


## ○ 交差検証の際、GBDTのearly stopping用eval_setはCVの中で指定すべき？それとも外？
early stopping機能を持ったGBDTをモデルとして交差検証を行う際、early stoppingの検証用データ(eval_set)を  
指定する方法は2通り考えられる。※下記を前提とする
* 前提1：　最終的な予測モデルにもearly stoppingを適用する。評価用データは学習データ全体に対してhold outで決める。
* 前提2：　CVインスタンスがあって、cv.split(データ)でfor文を回す場合を想定。cross_val_scoreは使わない。

メリット、デメリットを考えると方法2が良さそう。ネットやkaggleの例を見ても後者で実装していることが多い。  
Kaggleで与えられたデータだけで精度出したいのであれば方法１でもよいのかもしれないが、  
実務のようにeval_set用評価データが変わることも考慮すると、方法２でロバストなモデルであることを確認することが重要だと思われる。  
ただし、検証用foldに対するleakageは起きるので、高めの評価になることを留意しておく必要があると思う。  
leakageをどうしても抑えたい場合は、学習用foldをさらにhold outしてeval_set用データを分ける方法が考えられるが、  
CVの学習に使えるデータが少なくなってしまうことに注意。

* 方法1：　CVの前にeval_set（前提1の評価用データ）を定義しておき、CVのどのfoldに対しても同じeval_setで評価する
  * メリット：
    * eval_setが固定で良いので実装が楽。**fit_paramsでeval_set渡せば、cross_val_scoreを使うこともできる。
  * デメリット：
    * 全splitの結果が固定された評価用データに過学習気味になる。
    * CV前に定義したeval_setを使うときだけに有効な評価結果となるため、ロバストではない。実務ではeval_setが変わることも考えられる。
  <br></br>
* 方法2：　CVの中で分けられた学習用foldと検証用foldのうち、検証用foldをeval_setとして学習を行う。  
   ※lgb.cvでeval_setを渡さないとこの挙動になる。
  * メリット：
    * eval_setが毎回変わっても評価が安定していればロバストなモデルであることが確認できる。
  * デメリット
    * 各splitでの検証用foldに過学習気味の結果になる。（それでも上記の通り評価が安定していれば良いので、大きなデメリットではないかも）
    * 検証用foldをeval_setにしているので、一種のleakageになる。

## ○ 外れ値の検出方法
外れ値の検出は異常検知などにも応用されている。  
* https://recruit.gmo.jp/engineer/jisedai/blog/outlier_detection_quick_introduction/
* https://www.freecodecamp.org/news/how-to-detect-outliers-in-machine-learning/
* https://www.codexa.net/python-outlier/
* https://www.nli-research.co.jp/report/detail/id=42216?site=nli

## ○ 各特徴量の分布確認方法(ヒストグラム、KDE)
一般的にはヒストグラムが使われるが、KDEでその背後にある確率分布を推定して分布を確認するのも手。  
ヒストグラムは完全に手元にあるデータのみからカクカクの分布を見ることになるので、  
連続的に分布を確認することが出来ず、視覚的に細かく2つの分布を比較することが難しいという欠点があるが、  
KDEはbandwidthを適切に設定すれば滑らかな曲線で分布を確認することが出来る。KDEはsns.kdeplotで描ける。  
ただし、例えば目的変数のクラスごとに個別にKDEをプロットすると、異なる母集団から得られたデータとして  
クラスごとに分布がプロットされてしまうので、不適。この場合、hueにClassを指定するべき。  
こうすることで、最終的なKDEによる曲線に対するClassごとの内訳の分布(Kernel)を確認することが出来る。  

[KDE:カーネル密度推定](https://ja.wikipedia.org/wiki/%E3%82%AB%E3%83%BC%E3%83%8D%E3%83%AB%E5%AF%86%E5%BA%A6%E6%8E%A8%E5%AE%9A)

## ○ seaborn.regplotで外れ値の重みを軽くして回帰直線を引く方法
robust=Trueを渡す。  
https://seaborn.pydata.org/generated/seaborn.regplot.html

## 不均衡データの取り扱い
目的変数の陽性と陰性の割合が極端に違うような場合の分類タスクにおいて、  
これらをどう取り扱うかは評価指標に応じて適切な手法を選択する必要がある。  
主なものは以下。  
* アンダーサンプリング
  * 割合が高い方のデータ数を削減して、低い方に合わせる手法
* アンダーバギング
  * アンダーサンプリング結果に対してバギングを適用してバイアスを平準化する方法
* オーバーサンプリング
  * 割合が低い方のデータ数を増幅（KNN等を用いて新たなデータを作成する）して、高い方に合わせる手法
* 重みづけ
  * 分類器の予測結果に対してクラスの割合に応じた重みをつける。


参考： https://note.com/tatsu321/n/nf944d1165008